<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/A1_adapter_seq_bn_pretraining_1_percent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
Pretrains adapter

# Results

### Previous


# Setup

In [1]:
# Pretraining Dataset Settings
pretrain_dataset_name = "BigTMiami/amazon_split_25M_reviews_20_percent_condensed"
# 5% of 20% creates a 1% - I think this should take 15 minutes
pretrain_percentage = "5"
pretrain_train_split = f"train[:{pretrain_percentage}%]"
pretrain_eval_split = "validation"

# Adapter settings
experiment_tag = "A1"
adapter_short_desc = "pretrain"
adapter_type = "seq_bn" # could be "lora", etc.

adapter_hub_name = f"{experiment_tag}_adapter_{adapter_type}_{adapter_short_desc}_P_{pretrain_percentage}"
adapter_name = f"{experiment_tag}_adapter_{adapter_type}_{adapter_short_desc}_P_{pretrain_percentage}"

print(f"Adapter Hub Name: {adapter_hub_name}")

# Classification Dataset Settings
# classification_dataset_name = "BigTMiami/amazon_helpfulness"
# helpfulness_percentage = "5"
# classification_train_split = f"train[:{helpfulness_percentage}%]"
# classification_eval_split = "dev"

# Adapter to load from hub
# adapter_model_hub_name = "BigTMiami/adapter_classification_no_pretraining_20_percent"

# Classification Model Hub Name if pushed \ Output Dir
# classification_model_hub_name = "classification_model_hub_name"

Adapter Hub Name: A1_adapter_seq_bn_pretrain_P_5


In [2]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [3]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [4]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.1 MB/s eta 0:00:00


In [5]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [6]:
from datasets import load_dataset

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=[pretrain_train_split,pretrain_eval_split])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

Generating train split:   0%|          | 0/862683 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8360 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 43134
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 8360
})


In [7]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Add a new adapter
pretrain_model.add_adapter(adapter_name, config=adapter_type)

# Add head for masked language modeling
pretrain_model.add_masked_lm_head(adapter_name)

# Set the adapter to be used for training
pretrain_model.train_adapter(adapter_name)

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

# Train

In [11]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir=adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=11,
    weight_decay=0.01,
    logging_steps=200,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [12]:
from adapters import AdapterTrainer

pretrain_trainer = AdapterTrainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=data_collator,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [14]:
before_pretrain_eval_results = pretrain_trainer.evaluate()
before_pretrain_eval_results_loss = before_pretrain_eval_results["eval_loss"]
print(f"Before pretraining loss: {before_pretrain_eval_results_loss:.4f}")

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Before pretraining loss: 15.7952


In [15]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

Step,Training Loss


TrainOutput(global_step=122, training_loss=6.601088727106814, metrics={'train_runtime': 946.6442, 'train_samples_per_second': 45.565, 'train_steps_per_second': 0.129, 'total_flos': 1.1586342012715008e+16, 'train_loss': 6.601088727106814, 'epoch': 1.0})


In [16]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(after_pretrain_eval_results)

{'eval_loss': 4.290770053863525, 'eval_runtime': 91.1333, 'eval_samples_per_second': 91.734, 'eval_steps_per_second': 2.875, 'epoch': 1.0}


In [17]:
pretrain_model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=pretrain_dataset_name
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/A1_adapter_seq_bn_pretrain_P_5/commit/7c423b2a6601414314fbbdc6f87492d0c37abb2a', commit_message='Upload model', commit_description='', oid='7c423b2a6601414314fbbdc6f87492d0c37abb2a', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
print(f"Pretrain Before Loss: {before_pretrain_eval_results_loss}")
print(f"Pretrain After Loss: {after_pretrain_eval_results_loss}")

Pretrain Before Loss: 15.795221328735352
Pretrain After Loss: 4.290770053863525


In [19]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
